In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import ast
import tqdm


### Get all dirs and paths

In [ ]:
dataset_path = Path('../input/tensorflow-great-barrier-reef/') 
train_root_path = dataset_path / 'train_images'
train_paths = list(train_root_path.rglob('*.jpg'))

train_csv_path = dataset_path / "train.csv"
test_csv_path = dataset_path / "test.csv"

rgb2bgr = lambda x : cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
bgr2rgb = lambda x : cv2.cvtColor(x, cv2.COLOR_BGR2RGB)

In [ ]:
print (f"dirs in dataset root: {os.listdir(dataset_path)}")
print (f"sample train paths: {train_paths[:2]}")

### lets see what the csv's contain

In [ ]:
# train_df = pd.read_csv(train_csv_path)
# test_df = pd.read_csv(test_csv_path)

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

In [ ]:
# add image paths according to video name and frame number
train_df['path'] = train_df.apply(lambda row: '../input/tensorflow-great-barrier-reef/train_images/video_' 
                                  + str(row['video_id'])  + f"/{row['video_frame']}.jpg", axis = 1)
train_df['annotations'] = train_df['annotations'].apply(ast.literal_eval)
train_df

In [ ]:
train_df_valid_annots = train_df[train_df.apply(lambda x: len(x['annotations']) > 0, axis=1)]
train_df_valid_annots.reset_index(inplace = True, drop = True)
train_df_valid_annots

In [ ]:
test_df

### lets load a sample image 

In [ ]:
img = cv2.imread(train_df['path'][0])
print (f'image shape is {img.shape}')

plt.figure(figsize = (10, 10))
plt.imshow(bgr2rgb(img))

### Lets plot a bounding box for a single sample

In [ ]:
idx = 42
img = cv2.imread(train_df_valid_annots['path'][idx])
bboxs = train_df_valid_annots['annotations'][idx]

for bbox in bboxs:
    x1 = bbox['x']
    y1 = bbox['y']
    x2 = bbox['x'] + bbox['width']
    y2 = bbox['y'] + bbox['height']
    img = cv2.rectangle(img, [x1, y1], [x2, y2], [0, 0, 255], 10)
    
    
print (f'image shape is {img.shape}')

plt.figure(figsize = (10, 10))
plt.imshow(bgr2rgb(img))

### plotting bounding box stats

In [ ]:
train_df['num_bboxs'] = train_df['annotations'].apply(lambda x : len(x))

num_with_annots = np.count_nonzero(train_df['num_bboxs'])
num_without_annots = len(train_df) - num_with_annots

y_lbl = (num_with_annots, num_without_annots)
x_lbl = ['ANNOTATED IMAGES', 'NON-ANNOTATED IMAGES']

fig = plt.figure(figsize = (10, 5))
plt.bar(x_lbl, y_lbl)

plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.ylim(0, max(y_lbl) + 2000)

for i in range(len(y_lbl)):
    plt.annotate(str(y_lbl[i]), xy=(x_lbl[i],y_lbl[i]), ha='center', va='bottom', fontsize = 13)
    
plt.title('Annotated v/s Non Annotated Images', fontsize = 20);

**A lot of the images in the train dataframe seem to be unannotated, maybe we can use pseudo-labelling or similar techniques for the training dataset**

In [ ]:
bbox_list = []

for bboxs in train_df_valid_annots['annotations']:
    for bbox in bboxs:
        bbox_list.append(bbox)
        
bbox_df = pd.DataFrame(bbox_list)
bbox_df['aspect_ratio'] = bbox_df['width'] / bbox_df['height']
bbox_df['area'] = bbox_df['width'] * bbox_df['height']

In [ ]:
fig = plt.figure(figsize = (10, 5))
bbox_df['aspect_ratio'].hist(bins = 200)
plt.title('distribution of bbox aspect ratios', fontsize = 20)

Most of the bboxs seem to have a slightly greater width than height, but the bounding boxes in most cases don't seem to be thin/skewed

In [ ]:
fig = plt.figure(figsize = (10, 5))
bbox_df['area'].hist(bins = 200)
plt.title('distribution of bbox areas', fontsize = 20)

### Generating annotated videos

In [ ]:
os.makedirs('videos', exist_ok = True)

for video_id in tqdm.tqdm_notebook(set(train_df['video_id'])):
    temp_df = train_df[train_df['video_id'] == video_id]
    temp_df.reset_index(inplace = True)
    
    print (f'writing video {video_id}')
    
    sample_img = cv2.imread(temp_df['path'][0])
    
    frame_width = int(sample_img.shape[1])
    frame_height = int(sample_img.shape[0])
    frame_size = (frame_width,frame_height)
    fps = 10.
    
    output = cv2.VideoWriter(f'videos/video{video_id}.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, frame_size)

    for i, row in tqdm.tqdm_notebook(temp_df.iterrows(), leave = False, total = len(temp_df)):
        img = cv2.imread(row['path'])
        bboxs = row['annotations']
        
        for bbox in bboxs:
            x1 = bbox['x']
            y1 = bbox['y']
            x2 = bbox['x'] + bbox['width']
            y2 = bbox['y'] + bbox['height']
            
            img = cv2.rectangle(img, [x1, y1], [x2, y2], [0, 0, 255], 10)
        
        
        output.write(img)
        
    output.release()